In [8]:
import torch
import wandb
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR
from sklearn.model_selection import train_test_split
from torch.utils.data.sampler import SubsetRandomSampler


In [9]:
train_path = "/kaggle/input/nature-12k/inaturalist_12K/train"
test_path = "/kaggle/input/nature-12k/inaturalist_12K/val"
def load_data(train_path, val_split=0.2, batch_size=150, data_augmentation=False):
    transform_list = [transforms.Resize((224, 224)),transforms.ToTensor()]
    if data_augmentation:
        transform_list.insert(0, transforms.RandomHorizontalFlip())
        transform_list.insert(0, transforms.RandomRotation(10))

    transform = transforms.Compose(transform_list)

    # Load dataset
    dataset = ImageFolder(train_path, transform=transform)
    num_classes = len(dataset.classes)
    
    # Create stratified train-validation split
    train_indices, val_indices = train_test_split(list(range(len(dataset))), 
                                                   test_size=val_split, 
                                                   stratify=dataset.targets)

    # Create DataLoader for training set
    train_sampler = SubsetRandomSampler(train_indices)
    train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)

    # Create DataLoader for validation set
    val_sampler = SubsetRandomSampler(val_indices)
    val_loader = DataLoader(dataset, batch_size=batch_size, sampler=val_sampler)

    return train_loader, val_loader, num_classes

In [10]:
tl,vl,n = load_data(train_path, val_split=0.2, batch_size=1000, data_augmentation=False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [11]:
for batch_idx, (images, labels) in enumerate(vl):
    print(f"Batch {batch_idx + 1}:")
    print("Images shape:", images.shape)
    print("Labels shape:", labels.shape)

Batch 1:
Images shape: torch.Size([1000, 3, 224, 224])
Labels shape: torch.Size([1000])
Batch 2:
Images shape: torch.Size([1000, 3, 224, 224])
Labels shape: torch.Size([1000])


In [12]:
!wandb login 

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [13]:
class CNN(nn.Module):
    def __init__(self, input_channels, num_classes, num_filters, filter_size, dense_units, activation_conv='relu', data_augmentation=False, batch_norm=False, dropout=0.0, filter_organization='same'):
        super(CNN, self).__init__()

        self.conv_layers = nn.ModuleList()
        for i in range(5):
            if filter_organization == 'same':
                current_num_filters = num_filters
            elif filter_organization == 'double':
                current_num_filters = num_filters * (2 ** i)
            elif filter_organization == 'halve':
                current_num_filters = num_filters // (2 ** i)

            self.conv_layers.append(nn.Conv2d(input_channels, current_num_filters, filter_size, padding=1))
            if batch_norm:
                self.conv_layers.append(nn.BatchNorm2d(current_num_filters))
            if activation_conv == 'relu':
                self.conv_layers.append(nn.ReLU(inplace=True))
            elif activation_conv == 'gelu':
                self.conv_layers.append(nn.GELU())
            elif activation_conv == 'silu':
                self.conv_layers.append(nn.SiLU())
            elif activation_conv == 'mish':
                self.conv_layers.append(nn.Mish())
            self.conv_layers.append(nn.Dropout2d(dropout))
            self.conv_layers.append(nn.MaxPool2d(kernel_size=2))
            input_channels = current_num_filters

        input_size = 224  # Assuming input images are 224x224
        for _ in range(5):
            #input_size = (input_size // 2)
            input_size = (input_size - filter_size +3)
            input_size = (input_size - 2)//2 + 1
            
        self.fc = nn.Linear(input_channels * input_size * input_size, dense_units)
        self.out = nn.Linear(dense_units, num_classes)

    def forward(self, x):
        for layer in self.conv_layers:
            x = layer(x)

        x = torch.flatten(x, 1)
        x = torch.relu(self.fc(x))
        x = self.out(x)
        return x

In [14]:
sweep_config = {
    "method": "bayes",
    "name" : "CNN",
    "project": "DL_Assignment_2",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        "num_filters": {"values": [32, 64, 128]},
        "filter_size": {"values": [3,5,7]},
        "dense_units": {"values": [128, 256]},
        "activation_conv": {"values": ["relu", "gelu", "silu", "mish"]},
        "batch_size": {"values": [32,64]},
        "learning_rate": {"values": [ 0.0001]},
        "lr_decay": {"values": [0.1, 0.0005]},
        "num_epochs": {"value": 5},
        "data_augmentation": {"values": [True, False]},
        "batch_norm": {"values": [True, False]},
        "dropout": {"values": [0.2, 0.3]},
        "filter_organization": {"values": ["same", "double"]}
    }
}

In [15]:
def validate_model(model, val_loader, criterion, device):
    model.eval()  # Set model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    # Calculate average loss and accuracy
    avg_val_loss = val_loss / len(val_loader.dataset)
    val_accuracy = 100 * correct / total
    
    return avg_val_loss, val_accuracy

In [16]:
def train_model():
    wandb.init()
    config = wandb.config

    # Load data
    train_loader, val_loader, num_classes = load_data(train_path, val_split=0.2, batch_size=config.batch_size, data_augmentation=config.data_augmentation)

    # Define model
    model = CNN(input_channels=3, num_classes=num_classes, num_filters=config.num_filters,
                filter_size=config.filter_size, dense_units=config.dense_units, activation_conv=config.activation_conv,
                data_augmentation=config.data_augmentation, batch_norm=config.batch_norm, dropout=config.dropout,
                filter_organization=config.filter_organization)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
    scheduler = StepLR(optimizer, step_size=1, gamma=config.lr_decay)

    # Train the model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    for epoch in range(config.num_epochs):
        model.train()
        train_loss = 0.0
        correct_train = 0
        total_train = 0
        
        # Training loop
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * images.size(0)
            
            # Calculate training accuracy
            _, predicted = torch.max(outputs, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()
        
        # Calculate training loss and accuracy
        avg_train_loss = train_loss / len(train_loader.dataset)
        train_accuracy = 100 * correct_train / total_train

        # Validation
        avg_val_loss, val_accuracy = validate_model(model, val_loader, criterion, device)

        # Log metrics to wandb
        run_name = "{}_lr{}_batchsize{}_filter{}_dense{}_decay{}_dataAug{}_batchnorm{}_dropout{}_filterorg{}".format(config.activation_conv, config.learning_rate, config.batch_size, config.num_filters, config.dense_units, config.lr_decay,
                            config.data_augmentation, config.batch_norm,config.dropout,config.filter_organization)
        wandb.run.name = run_name
        wandb.log({"epoch": epoch+1, 
                   "train_loss": avg_train_loss, 
                   "train_accuracy": train_accuracy, 
                   "val_loss": avg_val_loss, 
                   "val_accuracy": val_accuracy})
        
        print("Epoch: {}, Train Loss: {:.4f}, Train Acc: {:.2f}%, Val Loss: {:.4f}, Val Acc: {:.2f}%".format(
            epoch+1, avg_train_loss, train_accuracy, avg_val_loss, val_accuracy))
        
        # Adjust learning rate
        scheduler.step()


In [6]:
train_loader, val_loader, num_classes = load_data(train_path, val_split=0.2, batch_size=64, data_augmentation=False)

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Load test data
test_dataset = ImageFolder(test_path, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)

# Define model
model = CNN(input_channels=3, num_classes=num_classes, num_filters=128,
            filter_size=5, dense_units=512, activation_conv="mish",
            data_augmentation=False, batch_norm=True, dropout=0.2,
            filter_organization="double")

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = StepLR(optimizer, step_size=1, gamma=0.5)

# Train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
for epoch in range(5):
    model.train()
    train_loss = 0.0
    correct_train = 0
    total_train = 0

    # Training loop
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)

        # Calculate training accuracy
        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    # Calculate training loss and accuracy
    avg_train_loss = train_loss / len(train_loader.dataset)
    train_accuracy = 100 * correct_train / total_train

    # Validation
#     avg_val_loss, val_accuracy = validate_model(model, val_loader, criterion, device)
    
    avg_test_loss, test_accuracy = validate_model(model, test_loader, criterion, device)

    # Log metrics to wandb
#     run_name = "{}_lr{}_batchsize{}_filter{}_dense{}_decay{}_dataAug{}_batchnorm{}_dropout{}_filterorg{}".format(config.activation_conv, config.learning_rate, config.batch_size, config.num_filters, config.dense_units, config.lr_decay,
#                         config.data_augmentation, config.batch_norm,config.dropout,config.filter_organization)
#     wandb.run.name = run_name
#     wandb.log({"epoch": epoch+1, 
#                "train_loss": avg_train_loss, 
#                "train_accuracy": train_accuracy, 
#                "val_loss": avg_val_loss, 
#                "val_accuracy": val_accuracy})

    print("Epoch: {}, Train Loss: {:.4f}, Train Acc: {:.2f}%, Test Loss: {:.4f}, Test Acc: {:.2f}%".format(
        epoch+1, avg_train_loss, train_accuracy, avg_test_loss, test_accuracy))

    # Adjust learning rate
    scheduler.step()


Epoch: 1, Train Loss: 1.7338, Train Acc: 23.23%, Test Loss: 2.0286, Test Acc: 27.20%
Epoch: 2, Train Loss: 1.5722, Train Acc: 30.63%, Test Loss: 1.9102, Test Acc: 33.15%
Epoch: 3, Train Loss: 1.5071, Train Acc: 33.29%, Test Loss: 1.8876, Test Acc: 34.15%
Epoch: 4, Train Loss: 1.4658, Train Acc: 36.07%, Test Loss: 1.8479, Test Acc: 35.75%
Epoch: 5, Train Loss: 1.4434, Train Acc: 37.43%, Test Loss: 1.8338, Test Acc: 36.30%


In [17]:
# Initialize and run the sweep
sweep_id = wandb.sweep(sweep_config, project="DL_Assignment_2")
wandb.agent(sweep_id, function=train_model,count=10)

Create sweep with ID: wsm0xqtz
Sweep URL: https://wandb.ai/cs23m025_dl_assignment_1/DL_Assignment_2/sweeps/wsm0xqtz


wandb: Agent Starting Run: 8zhspljb with config:
wandb: 	activation_conv: relu
wandb: 	batch_norm: True
wandb: 	batch_size: 64
wandb: 	data_augmentation: True
wandb: 	dense_units: 256
wandb: 	dropout: 0.3
wandb: 	filter_organization: double
wandb: 	filter_size: 3
wandb: 	learning_rate: 0.0001
wandb: 	lr_decay: 0.1
wandb: 	num_epochs: 5
wandb: 	num_filters: 64
wandb: Currently logged in as: cs23m025 (cs23m025_dl_assignment_1). Use `wandb login --relogin` to force relogin


Epoch: 1, Train Loss: 1.8075, Train Acc: 17.45%, Val Loss: 0.4290, Val Acc: 21.05%
Epoch: 2, Train Loss: 1.6843, Train Acc: 24.70%, Val Loss: 0.4260, Val Acc: 22.00%
Epoch: 3, Train Loss: 1.6689, Train Acc: 25.23%, Val Loss: 0.4267, Val Acc: 22.10%
Epoch: 4, Train Loss: 1.6627, Train Acc: 25.72%, Val Loss: 0.4255, Val Acc: 22.30%
Epoch: 5, Train Loss: 1.6696, Train Acc: 25.28%, Val Loss: 0.4277, Val Acc: 21.80%


epoch,▁▃▅▆█
train_accuracy,▁▇███
train_loss,█▂▁▁▁
val_accuracy,▁▆▇█▅
val_loss,█▂▃▁▆
epoch,5
train_accuracy,25.27816
train_loss,1.66959
val_accuracy,21.8
val_loss,0.42772


wandb: Agent Starting Run: et59ztj1 with config:
wandb: 	activation_conv: mish
wandb: 	batch_norm: False
wandb: 	batch_size: 64
wandb: 	data_augmentation: True
wandb: 	dense_units: 128
wandb: 	dropout: 0.3
wandb: 	filter_organization: same
wandb: 	filter_size: 7
wandb: 	learning_rate: 0.0001
wandb: 	lr_decay: 0.0005
wandb: 	num_epochs: 5
wandb: 	num_filters: 64


...

Epoch: 1, Train Loss: 1.7507, Train Acc: 20.29%, Val Loss: 0.4137, Val Acc: 27.25%
Epoch: 2, Train Loss: 1.6665, Train Acc: 26.09%, Val Loss: 0.4043, Val Acc: 28.30%
Epoch: 3, Train Loss: 1.6567, Train Acc: 26.25%, Val Loss: 0.4038, Val Acc: 29.35%
Epoch: 4, Train Loss: 1.6447, Train Acc: 26.19%, Val Loss: 0.4030, Val Acc: 28.50%
Epoch: 5, Train Loss: 1.6455, Train Acc: 26.90%, Val Loss: 0.4035, Val Acc: 29.35%


epoch,▁▃▅▆█
train_accuracy,▁▇▇▇█
train_loss,█▂▂▁▁
val_accuracy,▁▄█▅█
val_loss,█▂▂▁▁
epoch,5
train_accuracy,26.90336
train_loss,1.64545
val_accuracy,29.35
val_loss,0.40348


wandb: Agent Starting Run: 8xe33n5q with config:
wandb: 	activation_conv: relu
wandb: 	batch_norm: False
wandb: 	batch_size: 64
wandb: 	data_augmentation: True
wandb: 	dense_units: 256
wandb: 	dropout: 0.2
wandb: 	filter_organization: double
wandb: 	filter_size: 3
wandb: 	learning_rate: 0.0001
wandb: 	lr_decay: 0.0005
wandb: 	num_epochs: 5
wandb: 	num_filters: 64


Epoch: 1, Train Loss: 1.8414, Train Acc: 9.95%, Val Loss: 0.4544, Val Acc: 16.15%
Epoch: 2, Train Loss: 1.8268, Train Acc: 13.94%, Val Loss: 0.4544, Val Acc: 16.00%
Epoch: 3, Train Loss: 1.8244, Train Acc: 13.94%, Val Loss: 0.4544, Val Acc: 15.90%
Epoch: 4, Train Loss: 1.8254, Train Acc: 13.70%, Val Loss: 0.4543, Val Acc: 16.20%
Epoch: 5, Train Loss: 1.8243, Train Acc: 14.13%, Val Loss: 0.4543, Val Acc: 16.30%


epoch,▁▃▅▆█
train_accuracy,▁██▇█
train_loss,█▂▁▁▁
val_accuracy,▅▃▁▆█
val_loss,█▅▄▁▂
epoch,5
train_accuracy,14.12677
train_loss,1.8243
val_accuracy,16.3
val_loss,0.45435


In [ ]:
## part  b
import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torch.utils.data.sampler import SubsetRandomSampler

import wandb

In [ ]:
train_path = "/kaggle/input/nature-12k/inaturalist_12K/train"
test_path = "/kaggle/input/nature-12k/inaturalist_12K/val"
def load_data(train_path, val_split=0.2, batch_size=150, data_augmentation=False):
    transform_list = [transforms.Resize((224, 224)),transforms.ToTensor(),
                      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]
    if data_augmentation:
        transform_list.insert(0, transforms.RandomHorizontalFlip())
        transform_list.insert(0, transforms.RandomRotation(10))

    transform = transforms.Compose(transform_list)

    # Load dataset
    dataset = ImageFolder(train_path, transform=transform)
    num_classes = len(dataset.classes)
    
    # Create stratified train-validation split
    train_indices, val_indices = train_test_split(list(range(len(dataset))), 
                                                   test_size=val_split, 
                                                   stratify=dataset.targets)

    # Create DataLoader for training set
    train_sampler = SubsetRandomSampler(train_indices)
    train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)

    # Create DataLoader for validation set
    val_sampler = SubsetRandomSampler(val_indices)
    val_loader = DataLoader(dataset, batch_size=batch_size, sampler=val_sampler)

    return train_loader, val_loader, num_classes

In [ ]:
!wandb login "8506a95d562352ae82e224bf1d33b8cdf34c0c0e" 

In [ ]:
def pre_trained_model_run():
    wandb.init()
    train_loader, val_loader, num_classes = load_data(train_path, val_split=0.2, batch_size=32, data_augmentation=True)
    # Load pre-trained ResNet50 model
    pretrained_resnet50 = models.resnet50(pretrained=True)

    # Freeze all convolutional layers
    for param in pretrained_resnet50.parameters():
        param.requires_grad = False

    # Replace the classifier with a new fully connected layer

    pretrained_resnet50.fc = nn.Linear(pretrained_resnet50.fc.in_features, num_classes)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    learning_rate = 0.001
    optimizer = optim.Adam(pretrained_resnet50.fc.parameters(), lr=learning_rate)

    # Move model to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    pretrained_resnet50.to(device)

    # Train the model
    num_epochs = 10
    for epoch in range(num_epochs):
        # Training loop
        pretrained_resnet50.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)  # Move tensors to GPU
            optimizer.zero_grad()
            outputs = pretrained_resnet50(inputs)
            train_loss = criterion(outputs, labels)
            train_loss.backward()
            optimizer.step()

            running_loss += train_loss.item() * inputs.size(0)

            _, predicted = torch.max(outputs, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        avg_train_loss = running_loss / len(train_loader.dataset)
        train_accuracy = correct_train / total_train

        # Validation loop (evaluate model performance on validation dataset)
        pretrained_resnet50.eval()
        validation_loss = 0.0
        correct_val = 0
        total_val = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)  # Move tensors to GPU
                outputs = pretrained_resnet50(inputs)
                val_loss = criterion(outputs, labels)
                

                validation_loss += val_loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()
        validation_accuracy = correct_val / total_val
        avg_val_loss = validation_loss / len(val_loader.dataset)

        print(f'Epoch [{epoch+1}/{num_epochs}], '
              f'Train Loss: {avg_train_loss:.4f}, '
              f'Train Accuracy: {train_accuracy:.4f}, '
              f'Validation Loss: {avg_val_loss:.4f}'
              f'Validation Accuracy: {validation_accuracy:.4f}')

        run_name = "pre_trainded_{}_lr{}".format("adam", learning_rate)
        wandb.run.name = run_name
        wandb.log({"epoch": epoch+1, 
                    "train_loss": avg_train_loss, 
                    "train_accuracy": train_accuracy, 
                    "val_loss": avg_val_loss, 
                    "val_accuracy": validation_accuracy})

In [ ]:
sweep_config = {
     "method": "bayes",
    "name" : "pre_trained_model",
    "project": "DL_Assignment_2",
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        "learning_rate": {"values": [0.001]}
    }
}
sweep_id = wandb.sweep(sweep_config, project="DL_Assignment_2")
wandb.agent(sweep_id, function=pre_trained_model_run,count=1)